In [1]:
# Install dependencies
!pip install -q --upgrade keras_nlp keras
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.2/305.2 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 44.1 MB/s 

In [2]:
# Mount Google Drive
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from keras_nlp.models import GPT2CausalLMPreprocessor, GPT2CausalLM
import keras_nlp
import gradio as gr

Using TensorFlow backend


In [4]:
def generate_text(model, input_text, max_length=50):
    return model.generate(input_text, max_length=max_length)

In [5]:
preprocessor = GPT2CausalLMPreprocessor.from_preset(
    "gpt2_base_en",
    sequence_length=128,
)
gpt2_lm = GPT2CausalLM.from_preset(
    "gpt2_base_en",
    preprocessor=preprocessor,
)

100%|██████████| 448/448 [00:00<00:00, 700kB/s]
100%|██████████| 446k/446k [00:00<00:00, 10.8MB/s]
100%|██████████| 0.99M/0.99M [00:00<00:00, 18.1MB/s]
100%|██████████| 484/484 [00:00<00:00, 499kB/s]
100%|██████████| 475M/475M [00:08<00:00, 57.2MB/s]
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids


In [6]:
gpt2_lm.load_weights('/content/drive/MyDrive/Checkpoints/weights')

/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/task.py:47: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/task.py:47: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids


In [27]:
# Gradio app
# <a href="https://www.freepik.com/icon/user_456212#fromView=search&term=avatar&track=ais&page=1&position=22&uuid=48125587-eeb5-4fe3-9eb2-f9fe7330f4fe">Icon by Freepik</a>
# <a href="https://www.freepik.com/icon/ai_2814666#fromView=search&term=robot&track=ais&page=1&position=20&uuid=58780fb9-dab6-4fb1-9928-479b2926a242">Icon by Freepik</a>

theme = gr.themes.Soft().set(
    background_fill_primary='white',
    background_fill_primary_dark='white',
)
with gr.Blocks(theme=theme,css="""
    .gradio-container {
        background-color: white;
        width: 70vw;
    }
    #chatbot{
        background-image: url("https://png.pngtree.com/thumb_back/fh260/background/20201014/pngtree-breast-cancer-awareness-pink-ribbons-background-design-image_417234.jpg");
    }
    #chatbot .bubble-wrap::-webkit-scrollbar {
        width: 20px;
    }

    #chatbot .bubble-wrap::-webkit-scrollbar-thumb {
        background-color: whitesmoke;
        border-radius: 20px;
        border: 6px solid transparent;
        background-clip: content-box;
    }

    #chatbot .bubble-wrap::-webkit-scrollbar-thumb:hover {
        background-color: grey;
    }

    #chatbot .bubble-wrap::-webkit-scrollbar-track {
        background-color: transparent;
    }
    #chatbot .message p{
        text-align: start;
        color: white;
    }
    h1, p {
        text-align: center;
        color: black;
    }
    body #footer_note {
        text-align: center;
        font-size: x-small;
        font-weight:bold;
    }
    .label {
        display:none;
    }
    textarea, .gallery-item, .gallery-item:hover {
      color: black;
      border: 1px black solid;
      background-color: white;
    }
    .user {
      background-color: #374151;
    }
    .user {
      background-color: #111827;
    }
    .gallery-item:hover {
      color: white;
      border: 1px black solid;
      background-color: black;
    }
    body gradio-app  {
      background-color: white;
    }
    """) as demo:
    gr.HTML(f"""
        <html>
        <body>
            <h1>Welcome, I'm CancerBot 🤖</h1>
            <p>Here you can ask all questions about cancer</p>
        </body>
        </html>
    """)

    def return_message(message, history, model=gpt2_lm, max_length=128):
        if len(message) <= 1:
            gr.Warning('Please enter a message with more than one character.')
        elif len(message) > max_length:
            gr.Warning(f"Input should not exceed {max_length} characters.")
        else:
            cancer_answer = generate_text(model, message)
            message = "**You**\n" + message
            history.append([message, f"**CancerBot**\n{cancer_answer}"])
        return "", history

    chatbot = gr.Chatbot(
        height="60vh",
        bubble_full_width=True,
        avatar_images=(["/content/drive/MyDrive/Data/avatar.png", "/content/drive/MyDrive/Data/robot.png"]),
        show_copy_button=True,
        likeable=True,
        layout='bubble',
        elem_id='chatbot',
        show_label=False,
    )
    with gr.Row():
        input_box = gr.Textbox(placeholder="Message CancerBot...", container=False, scale=9)
        submit_btn = gr.Button(value="⬆", scale=1)
        submit_btn.click(return_message, [input_box, chatbot],[input_box, chatbot])
    examples = gr.Examples(examples=["What is a thyroid cancer ?", "How can I know that I have a lung cancer ?",
			"How many types of cancer ?"], inputs=[input_box], label="")
    input_box.submit(return_message, [input_box, chatbot],[input_box, chatbot])
    gr.HTML(f"""
            <html>
            <body>
            <p id="footer_note">CancerBot is based on cancer documents. Consider checking important information.</p>
            </body>
            </html>
            """)
demo.queue(default_concurrency_limit=34) # 32 students, 2 teachers
demo.launch(share=True,favicon_path="/content/drive/MyDrive/Data/robot.png")

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://866ca59b38d9bbfb07.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
